In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## maze.py

La llibreria auxiliar ```maze.py```, conté les funcions necessàries per crear i dibuixar laberints aleatoris. A continuació destaquem les funcions més rellevants tot i que podeu obrir el fitxer i investigar.

- ```Maze(x_num, y_num, seed=None, p_down=0.2, p_build=0)```: Aquesta funció construeix i retorna un laberint aleatori de dimensió $x_\text{num} \times y_\text{num}$. El paràmetre $\textit{seed}$ permet fixar una llavor de forma que sempre es generi el mateix laberint. És interessant que, un cop hagueu programat la solució dels exercicis, proveu de modificar-lo per veure que funciona en diferents casos. Els paràmetres $p_\text{down}$ i $p_\text{build}$ són probabilitats de destrucció d'una paret o de construcció. Així, si $p_\text{down}$ és proper a 1, s'eliminen gairebé totes les parets del laberint, creant molts camins entre totes les cel·les. En canvi, si $p_\text{build}$ és proper a 1, es crearan parets que faran que el laberint no tingui solució per a gairebé cap parella de cel·les.

- ```maze.display_maze(height=200, plot_path=False)```: Com el seu nom indica, ens mostra el laberint que hem creat. Podem definir l'alçada del dibuix i l'amplada s'ajusta automàticament. Com veurem, podem assignar un camí a cada laberint. Si hi ha un camí assignat, el podem mostrar cridant la funció amb la variable _plot\_path = True_.

- ```maze.maze_graph()```: Aquesta funció, que no rep cap paràmetre per entrada, ens retorna un graf de la classe nx.Graph() que representa el laberint.

- ```maze.set_path(path)```: Donat un camí definit com una sequència de nodes, el podem assignar al laberint usant aquesta funció. Recorda que si vols visualitzar un camí assignat, pots fer-ho cridant a ```display_maze(plot_path=True)```.

In [2]:
from matplotlib import pyplot as plt
import networkx as nx
from maze import *

<div class="alert alert-success">
<h1>Problema 1: BFS (2,5p)</h1>
<p>
    Implementeu l'algorisme <b>BFS</b>. Donat un graf de la llibreria networX, la funció <b>bfs()</b> ha de retornar les caselles accessibles des del paràmetre <b>origin</b>, incloent-hi aquesta, en l'ordre d'exploració. És a dir, la llista retornada sempre inclourà, com a mínim, el node orígen.<br>
    Recordeu que és necessari <b>no canviar l'estructura de la funció</b>. No canvieu ni el nom de la funció, ni el nom dels paràmetres d'entrada ni els de sortida. <br>
</p>    
    
</div>

In [6]:
def bfs(G, origin):
    """
    Params
    ======    
    :G: Graf. Ha de ser un objecte de la classe nx.Graph
    :origin: Índex del node orígen
    
    Returns
    =======
    :visited: El conjunt de nodes visitats durant l'exploració BFS en l'ordre en que han estat explorats
    """
    visited = []        #Els nodes visitats en ordre
    visited2 = set()    #Els nodes visitats sense ordre per poder fer la comprovació eficientment

    pending_nodes = []  
    pending_nodes.append(origin)

    #Fem un recorregut per tots els nodes accessibles
    
    while len(pending_nodes) > 0:
        current_node = pending_nodes.pop(0)

        #Si encara no hem visitat el node l'afegim a les llistes i 
        #posem els seus veïns a la llista de nodes pendents de visitar

        if current_node not in visited2:
            visited.append(current_node)
            visited2.add(current_node)

            for nei in G.neighbors(current_node):
                pending_nodes.append(nei)

    
    return visited

In [ ]:
# Obtenim el graph associat al laberint on cada node es una cel·la i tindrem una aresta si entre dues cel·les no existeix paret
maze = Maze(7, 7, p_build=0.2, seed=8)
G = maze.maze_graph()

A = 24                    
visited = bfs(G, A)
num = len(visited)

print(f"Des de la cel·la {A} podem accedir a {num} caselles")
print(f"Ordre d'exploració: {visited}")
maze.set_values(visited, 1, color=(255,0,0))
maze.set_value(A, 1, color=(0,255,0))

maze.display_maze(plot_path=True, height=200)

In [ ]:
# Obtenim el graph associat al laberint on cada node es una cel·la i tindrem una aresta si entre dues cel·les no existeix paret
maze = Maze(50, 7, p_build=0.2, seed=8)
G = maze.maze_graph()

A = 174                      
visited = bfs(G, A)
num = len(visited)

print(f"Des de la cel·la {A} podem accedir a {num} caselles")
print(f"Ordre d'exploració: {visited}")
maze.set_values(visited, 1, color=(255,0,0))
maze.set_value(A, 1, color=(0,255,0))

maze.display_maze(plot_path=True, height=150)

<div class="alert alert-success">
<h1>Problema 2: Holes (2,5p)</h1>
<p>
    Implementeu una funció anomenada <b>holes()</b> que, donat un graf, un node orígen, un node destí, i un diccionari de nodes amb penalitzacions, trobi el camí més curt entre orígen i destí.<br>
    Si el camí passa per un dels nodes del diccionari, el cost de visitar el node serà la penalització en comptes de ser cost 1.<br>
    Heu de basar la vosta solució en l'algorisme <b>Dijkstra</b>.
</p>    
    
</div>

In [9]:
import queue

def holes(G, origin, destination, holes_dct={}):
    """
    Params
    ======
    :G: Graf del qual en volem extreure el camí mínim. Ha de ser un objecte de la classe nx.Graph
    :origin: Índex del node orígen
    :destination: Índex del node destí
    :holes_dct: Un diccionari del tipus {node: penalització}. Passar per cada node del diccionari té un cost diferent a 1.
    
    Returns
    =======
    :path: Una llista de nodes del camí més curt entre els nodes 'origin' i 'destination' (ambdós inclosos).
    :cost: Un enter amb el cost de recórrer el camí, incloent-hi les penalitzacions.
    """
    
    path = []                   #Llista per ficar el camí més curt
    cost = 0                    #Cost total del camí 
    pq = queue.PriorityQueue()  #Cua de prioritat per agafar els nodes de menor cost
    pq.put((0, origin))         
    anterior = {origin: None}   #Diccionari per veure el node que has visitat anteriorment per arribar a dit node
    cost_list = {origin: 0}     #Diccionari per guardar el cost mínim per arribar a cada node
    current_cost = 0            #On posarem el cost que costa arribar a l'actual node
    current_node = None

    #O( (V+E) * log(V) )

    while not pq.empty():   #El while s'executa V vegades

        #S'agafa un node i el seu cost del que menys ens ha costat arribar a la cua 
        current_cost, current_node = pq.get()   #O(log(V))

        #Si el node actual és el destí sortim per posar el camí fet
        if current_node == destination:
            break

        #Es calcula un nou cost per arribar als seus veïns i si es menor al que ja tenien
        #o no tenien cost s'actualitza i es fica a "anterior" per tal de anar fent el camí
        
        for nei in G.neighbors(current_node):   #Pel bucle del while tot el que esta a dins d'aquest for es fa E cops
            new_cost = current_cost + holes_dct.get(nei, 1)

            if nei not in cost_list or new_cost < cost_list[nei]:
                cost_list[nei] = new_cost
                pq.put((new_cost, nei)) #O(log(V))
                anterior[nei] = current_node


    #Es reconstrueix el camí fet desde el destí i després s'inverteix la llista
    current_node = destination

    while current_node is not None:
        path.append(current_node)
        current_node = anterior[current_node]
    
    path.reverse()
    
    #Obtenim el cost per arribar al destí
    cost = cost_list[destination]

    return path, cost


In [ ]:
# Proveu el vostre algorisme!

maze = Maze(30, 10, seed=15)

# Obtenim el graph associat al laberint on cada node es una cel·la i tindrem una aresta si entre dues cel·les no existeix paret
G = maze.maze_graph()

A = 0
B = len(G.nodes)-1
color = (255, 0, 255)

h_dcts = [
    {}, # Sense penalització
    {8:20,72:2, 227:10},
    {8:15, 72:10, 227:9},
]
for holes_dct in h_dcts:
    maze.set_values(list(holes_dct.keys()), 1, color)
    path, cost = holes(G, A, B, holes_dct)

    maze.set_path(path)
    print(f"Cost total: {cost}")
    print('Longitud del camí:', len(path))
    maze.display_maze(plot_path=True, height=200)

    maze.set_values(list(holes_dct.keys()), 0, color)

<div class="alert alert-success">
<h1>Problema 3: Checkpoint (2,5p)</h1>
<p>
    Implementeu una funció anomenada <b>checkpoint()</b> que, donat un graf, un node origen, un node destí i un node extra, trobi el camí més curt des del node origen fins el node destí passant pel node extra. Com en l'exercici anterior, tindrem un diccionari de punts on aplicarem una penalització.<br>
</p>    
    
</div>

In [11]:
#m <= V --> O( (V+E) * log(V) )
def checkpoint(G, origin, destination, extra, holes_dct={}):
    """
    Params
    ======
    :G: Graf del qual en volem extreure el camí mínim. Ha de ser un objecte de la classe nx.Graph
    :origin: Índex del node orígen
    :destination: Índex del node destí
    :extra: Índex d'un node extra per on ha de passar el camí
    :holes_dct: Un diccionari del tipus {node: penalització}. Passar per cada node del diccionari té un cost diferent a 1.
    
    Returns
    =======
    :path: Una llista de nodes del camí més curt entre els nodes 'origin' i 'destination' que passa per 'extra'.
    :cost: Un enter amb el cost de recórrer el camí, incloent-hi les penalitzacions.
    """
    path = []
    path2 = []
    cost = 0
    cost2 = 0

    # 2 * [(V+E) * log(V)] = O((V+E) * log(V))

    #Fem el camí més óptim del node origen al node extra i el sumem al camí més óptim del node extra al node destí.

    path, cost = holes(G, origin, extra, holes_dct)
    path2, cost2 = holes(G, extra, destination, holes_dct)

    #O(m) donde m es el nº de elementos de path2
    path.extend(path2)
    #O(1)
    cost += cost2
    
    return path, cost

In [ ]:
# Proveu el vostre algorisme!

# Creem un laberint i n'extraiem el graph
maze = Maze(6,6, seed=17)
G = maze.maze_graph()

# Definim l'inici, el fi, un node extra i la llista amb penalització
A = 30
B = len(G.nodes)-1
E = 8

h_dcts = [
    {}, # Sense penalització
    {13:3, 28:5},
    {13:10, 28:20},
]
for holes_dct in h_dcts:
    # La funció set_value ens permet 'activar' un node per a que surti dibuixat al laberint.
    maze.set_value(E, 1)                         # Checkpoint
    maze.set_values(list(holes_dct.keys()), 1, (255,0,255))  # Penalitzacions

    path, cost = checkpoint(G, A, B, E, holes_dct)
    print(f'El cost del camí més curt que passa per E és: {cost}')
    print('La distàcia del camí més curt que passa per E és:', len(path))
    print(f'El camí és: {path}')
    maze.set_path(path)
    maze.display_maze(plot_path=True, height=200)
    
    maze.set_value(E, 0)                         # Checkpoint
    maze.set_values(list(holes_dct.keys()), 0, (255,0,255))  # Penalitzacions

<div class="alert alert-success">
<h1>Problema 4: Checkpoints list (2,5p)</h1>
<p>
    Implementeu una funció anomenada <b>checkpoints_list()</b> que, donat un graf, un node origen, un node destí i una llista de nodes extres, trobi el camí més curt des del node origen fins el node destí passant per tots els nodes extra, en l'ordre més òptim. Com en l'exercici anterior, tindrem un diccionari de punts on aplicarem una penalització.<br>
    <b>Ajuda</b>: Podeu fer servir la funció <b>itertools.permutations</b>.
</p>    
    
</div>

In [13]:
import itertools
def checkpoints_list(G, origin, destination, extras, holes_dct={}):
   """
   Params
   ======
   :G: Graf del qual en volem extreure el camí mínim. Ha de ser un objecte de la classe nx.Graph
   :origin: Índex del node orígen
   :destination: Índex del node destí
   :extras: Llista d'índexs de nodes per on ha de passar el camí.
   :holes_dct: Un diccionari del tipus {node: penalització}. Passar per cada node del diccionari té un cost diferent a 1.
   
   Returns
   =======
   :path: Una llista de nodes del camí més curt entre els nodes 'origin' i 'destination' que passa per tots els nodes 'extras'.
   :cost: Un enter amb el cost de recórrer el camí, incloent-hi les penalitzacions.
   """

   path = []
   cost = float('inf')
   posibilities = itertools.permutations(extras)   #Creem totes les posibilitats de visitar els nodes extres.
   calculated_cost = {} #Diccionari on anem guardant els camins i costos que anem fent per tal de no tornar a calcular-ho.

   #De totes les posibilitats d'ordre de visitar els nodes extres agafem una.

   for posibility in posibilities:
      total_path = []
      total_cost = 0
      current_node = origin

      #Anem fent el camí node per node.

      for node in posibility:

         #Comprovem si el camí que hem de calcular ara ja l'hem fet, en cas que si,
         #no cridem a holes sino que agafem el camí i el cost que ja hem calculat.

         if (current_node,node) in calculated_cost:
            inter_path, inter_cost = calculated_cost[(current_node,node)]
         else:
            inter_path, inter_cost = holes(G,current_node,node,holes_dct)  #O((V+E) * log(V))
            calculated_cost[(current_node,node)] = (inter_path,inter_cost)

         total_path += inter_path
         total_cost += inter_cost
         current_node = node  

      #Afegim el camí de l'últim node fins al node destí.

      if (current_node, destination) in calculated_cost:
         inter_path, inter_cost = calculated_cost[(current_node, destination)]
      else:
         inter_path, inter_cost = holes(G, current_node, destination, holes_dct)
         calculated_cost[(current_node, destination)] = (inter_path, inter_cost)

      total_path += inter_path
      total_cost += inter_cost

      #Si el que ens ha costat fer aquesta posibilitat és la menor fins ara actualitzem els valors que retornarem.

      if total_cost < cost:
         path = total_path
         cost = total_cost

   return path, cost

In [ ]:
# Proveu el vostre algorisme!

# Creem un laberint i n'extraiem el graph
maze = Maze(30, 10, seed=25)
G = maze.maze_graph()

# Definim l'inici, el fi i node extres
A = 20
B = len(G.nodes)-1
E = [10,187, 286, 27]

h_dcts = [
    {}, # Sense penalització
    {216:200, 208:30, 84:50},
    {79:500, 127:100, 84:5, 15: 500}
]
for holes_dct in h_dcts:
    # La funció set_values ens permet 'activar' nodes per a que surtin dibuixat al laberint.
    maze.set_values(E, 1)                        # Checkpoint
    maze.set_values(list(holes_dct.keys()), 1, (255,0,255))  # Penalitzacions

    path, cost = checkpoints_list(G, A, B, E, holes_dct)
    print(f'El cost del camí més curt que passa per E és: {cost}')
    print('La distàcia del camí més curt que passa per E és:', len(path))
    maze.set_path(path)
    maze.display_maze(plot_path=True, height=200)

    # De la mateixa forma la funció set_values també ens permet 'desactivar' els nodes.
    maze.set_values(E, 0)
    maze.set_values(list(holes_dct.keys()), 0, (255,0,255))

In [ ]:
plt.figure(figsize=(20,8))

colorA, colorB, colorE, colorP, colorPTH, colorDEF = '#00ff00', '#0000ff', '#00ffff', '#ff00ff', '#ff0000', '#1f78b4'
color_map = [colorA if n==A else colorB if n==B else colorE if n in E else colorP if n in holes_dct else colorPTH if n in path else colorDEF for n in G.nodes]

pos = nx.kamada_kawai_layout(G).items()

pos = {k: v[::-1] for k,v in pos}

# Dibuixem el graf definint la posició, el color i la mida de cada node.
nx.draw(G, pos=pos, node_color=color_map, node_size=100)